# Proyecto 1 - Etapa 1

In [12]:
import pandas as pd
import nltk

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score

from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import tree
import sklearn as sklearn
from joblib import dump, load
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer

In [13]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('stopwords-es')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/paulinaarrazola/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/paulinaarrazola/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading stopwords-es: Package 'stopwords-es' not
[nltk_data]     found in index
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/paulinaarrazola/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# 1. Carga de datos

In [14]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)
news=pd.read_csv("./fake_news.csv", sep=";")
data= news.copy()

In [15]:
data.shape

(57063, 5)

In [16]:
data.sample(5)

,ID,Label,Titulo,Descripcion,Fecha
51307,ID,0,La Mesa de la Asamblea de Madrid aprueba que C...,Toda la oposición había solicitado esta inicia...,22/03/2018
15995,ID,0,Celaá critica al Iniciativa vers per Catalunya...,"La demanda social es un criterio que, desde mi...",02/03/2019
14887,ID,1,La derecha coincide en su estrategia contra la...,El efecto llamada es el comodín que utiliza la...,09/12/2020
31685,ID,0,José Primo de Rivera asume Política Territoria...,La nueva ministra de Política Territorial y po...,10/07/2021
37218,ID,1,Madrid vira a la derecha y Barcelona avanza ha...,PP y Ciudadanos miran a Vox para hacerse con e...,28/05/2019


Como podemos observar, nos encontramos con cinco variables dentro del dataset. En primer lugar, encontramos una columna "Label" la cual si bien no esta bien definido el nombre podemos concluir que indica la veracidad del registro asociado. Luego, hallamos variables de informacion sobre la noticia (Titulo, Descripcion y Fecha). Finalmente, tenemos la columna "ID", que en todas las filas analizadas aparece simplemente como "ID", en lugar de un identificador único. Esto sugiere que la columna podría no aportar valor al análisis y que probablemente deba ser eliminada para evitar información redundante o errónea.

# 2. Entendimiento de datos

In [17]:
data.dtypes

ID             object
Label           int64
Titulo         object
Descripcion    object
Fecha          object
dtype: object

no se realiza ".describe" porque no tiene sentido en este caso porque como todos los datos son de categoricos menos el id no tiene sentido 

In [18]:
data.isnull().sum()

ID              0
Label           0
Titulo         16
Descripcion     0
Fecha           0
dtype: int64

Se decide no quitar aquellos registros sin titulo dado que las descripciones pueden contener información valiosa, ademas, se pueden identificar las noticias bajo su id o manejar los valores nulos de otra manera como relleno o NLP 

In [19]:
#Saber cuantas de las noticias que tenemos son falsas. DESCRIBIR RESULTADOS
porcentajes = data['Label'].value_counts(normalize=True) * 100
print(porcentajes)

Label
1    58.107706
0    41.892294
Name: proportion, dtype: float64


In [20]:
""" from nltk.probability import FreqDist
import random
import matplotlib.pyplot as plt

# Select 5 random indices within the valid range
random_indices = random.sample(range(data.shape[0]), 5)
text = ' '.join(data['Descripcion'][i] for i in random_indices)

fdist = FreqDist(text.split())

plt.figure(figsize=(12, 6))  
fdist.plot(25, cumulative=False)
plt.show() """

from nltk.corpus import stopwords

stop_words = set(stopwords.words('spanish'))
words = [word for word in text.split() if word.lower() not in stop_words]

fdist = FreqDist(words)
plt.figure(figsize=(12, 6))
fdist.plot(25, cumulative=False)
plt.show()


NameError: name 'text' is not defined

Se seleccionan 5 descripciones de registros al azar para encontrar las palabras más comunes en estos pedazos y tener un primer entendimiento de las palabras más predominantes que podemos llegar a encontrar, tambien se realiza un filtro para evitar articulos y preposiciones y asi generar palabras mas significativas. Como podemos observar, las palabras que mas se resaltan estam relacionadas con politica. 

In [21]:
data['Descripcion'].describe()

count                                                 57063
unique                                                49638
top       La Fundación En Acción ha participado con Públ...
freq                                                      7
Name: Descripcion, dtype: object

In [22]:
data.nunique()

ID                 1
Label              2
Titulo         51604
Descripcion    49638
Fecha           2271
dtype: int64

Resgistros totales: 57063
Registros unicos: 49638
13% de las descripciones registros son duplicados
Hay títulos que comparten la misma descripcion

In [23]:
# Buscamos registros vacios
empty_texts = data[data['Descripcion'].str.strip() == '']
print(f"Descripciones vacias: {len(empty_texts)}")

Descripciones vacias: 0


# 3. Procesamiento

### 3.1 Limpieza de datos

Se eliminan los registros de las descripciones que estan repetidos porque el motor se puede confundir o puede aumentar el bias (explciar mejor por que). Se deja la primera ocurrencia. Se elimina duplicados por que no coinciden los duplicados entre columnas y hay algunas que si tiene sentido la duplicidad (la fecha)

In [24]:
data = data.drop_duplicates(subset=['Descripcion'], keep='first')
#confirmamos que se hayan eliminado los duplicados
data.shape

(49638, 5)

In [25]:
import re
import unicodedata

#Se hace una isnpeccion visual para identificar caracteres especiales que reemplazan otra letra y se incluyen en el diccionario 
# para corregirlo y no perder esas palabras cuando eliminemos los caracteres especiales

mapa_reemplazo= {
    "√°": "a",
    "√©": "e",
    "√≠": "i",
    "√≥": "o",
    "√º":"ú",
    "√±": "ñ",
}

def correct_common_replacements(text):

    for wrong, right in mapa_reemplazo.items():
        text = text.replace(wrong, right)
    return text

def remove_non_ascii(words):
    
    new_words = []
    for word in words:
        if word is not None:
          new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
          new_words.append(new_word)
    return new_words

def to_lowercase(words):

    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
  
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_integers(words):
  
    new_words = []
    for word in words:
        new_word = re.sub(r"\b\d+\b", "", word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words, stopwords=stopwords.words('spanish')):
    new_words = []
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words

def preprocessing(words):
    text = ' '.join(words)
    text = correct_common_replacements(text)
    words = text.split()
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_integers(words)
    words = remove_stopwords(words)
    return words


Se realiza una correción de caracteres especiales ya que algunas codificaciones defectuosas pueden hacer que caracteres acentuados se conviertan en símbolos extraños (√°, √©, etc.). Luego, Ese hace eliminación de caracteres no ASCII convirtiendo caracteres con tildes en su versión sin tilde utilizando, esto es útil porque en algunos modelos, la misma palabra con y sin tilde podría ser tratada como diferente. Después,  se hace eliminación de puntación, números, palabras vacias y conversión a minusculas. Se realizan estas funciones con el objetivo de mejorar la calidad del texto antes del análisis, reducir el ruido eliminando información irrelevante y evitar problemas de codificación con caracteres especiales.


EXPLICAR TODOS LOS PASOS ANTERIORES (ESPECIFICAMENTE POR QUE SE HACEN MAS QUE COMO)

### 3.2 Tokenización

Se realiza tokenización para estandatizar el analisis, eliminar el ruido y poder representar el texto de mejor manera. Es importante denotar que se hicieron uso de las funciones previemente observadas para la reduccion de ruido. Es imprecindible este paso para mejorar la calidad de los datos antes de analizarlos o usarlos en los modelos.

EXPLICAR NECESIDAD Y DECIR QUE SE LE APLICAN LAS FUNCIONES DE ARRIBA PARA REDUCIR EL RUIDO (DESARROLLAR)

In [26]:
import contractions

data['Descripcion'] = data['Descripcion'].apply(contractions.fix)

Se realiza una expansión de contracciones con el fin de evitar la perdida de información al convertir palabras contraídas en su versión completa, mejorar la calidad de los datos para modelos y reducir la ambiguedad en el analisis.

In [27]:
data['words'] = data['Descripcion'].apply(word_tokenize).apply(preprocessing)
data.head()

,ID,Label,Titulo,Descripcion,Fecha,words
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[diario, britanico, publico, pasado, jueves, e..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[revelan, gobierno, negocio, liberacion, mirel..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[valencianismo, convoca, castello, fiesta, gra..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[politica, igual, negociar, empresarios, negoc..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[entrevista, punt, avui, lider, jxcat, desdram..."


Se tokeniza el campo descripcion para convertir el texto en una lista de palabras y se aplica la funcion preprocessing() para limpiar y normalizar cada token. Como podemos ver en la columna "words", contiene una versión limpia y lista del texto para análisis facilitando tareas como detección de noticias falsas

### 3.3 Normalización

La normalización es un paso clave en este caso para el procesamiento de lenguaje natural, ya que reduce la variabilidad del texto sin perder su significado, lo que ayuda a mejorar la calidad del análisis. Cuando se trabaja con datos de texto, se pueden presentarse múltiples variaciones de una misma palabra que deben tratarse como equivalentes.

EXPLICAR NECESIDAD Y PROCESO

In [28]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = SnowballStemmer('spanish')
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

data['words'] = data['words'].apply(stem_and_lemmatize) #Aplica lematización y eliminación de prefijos y sufijos.
data.head()

,ID,Label,Titulo,Descripcion,Fecha,words
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[diari, britan, public, pas, juev, editorial, ..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[revel, gobiern, negoci, liber, mirel, cambi, ..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[valencian, convoc, castell, fiest, grand, con..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[polit, igual, negoci, empresari, negoci, grup..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[entrev, punt, avui, lid, jxcat, desdramatiz, ..."


Se aplica stemming y lemmatization a las palabras de la columna 'words' del dataset con el objetivo de reducir las palabras a su forma base.

La función stem_words(words) usa SnowballStemmer para recortar palabras eliminando sufijos y prefijos. Es importante entender que el stemming puede generar palabras sin sentido, sin embargo, reduce la dimensionalidad del texto.

La función lemmatize_verbs(words) usa WordNetLemmatizer para convertir verbos a su forma base.



### 3.4 Selección de campos

Acá si explicar pasito a pasito en palabras cortas que se hizo. Ver ejemplo en gitlab

In [29]:
data['words'] = data['words'].apply(lambda x: ' '.join(map(str, x)))
data

,ID,Label,Titulo,Descripcion,Fecha,words
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,diari britan public pas juev editorial proxim ...
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,revel gobiern negoci liber mirel cambi otorg p...
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,valencian convoc castell fiest grand conmemor ...
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,polit igual negoci empresari negoci grup parla...
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,entrev punt avui lid jxcat desdramatiz posibl ...
...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,gobiern regional indic atencion dia inclu aten...
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,si higien democrat llev exigent ten pas person...
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,coordin federal iu asegur monarqui putrefact c...
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,santiag abascal vox derech deb form gobiern ca...


In [31]:
X_train, X_test, y_train, y_test = train_test_split(data["Descripcion"], data["Label"], test_size = 0.2, stratify = data["Label"], random_state = 1)

# 4. Modelos

### 4.1 Random Forest (Paulina Arrázola)